### Load project

In [1]:
from pathlib import Path

from mynd.backend import metashape as metashape
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


CACHE: Path = Path("/data/kingston_snv_01/acfr_point_clouds")
INPUT_DIR: Path = Path("/data/kingston_snv_01/acfr_metashape_projects_dev")
OUTPUT_DIR: Path = Path(
    "/data/kingston_snv_01/acfr_metashape_projects_registered"
)


# NOTE: Change to dense project
INPUT_PROJECT: Path = INPUT_DIR / "r234xgje_dense_with_metadata.psz"
DESTINATION_PROJECT: Path = OUTPUT_DIR / "r234xgje_registered_with_metadata.psz"

assert (
    INPUT_PROJECT != DESTINATION_PROJECT
), "source and destination project cannot be the same!"

assert CACHE.exists(), f"directory does not exist: {CACHE}"
assert OUTPUT_DIR.exists(), f"directory does not exist: {OUTPUT_DIR}"
assert INPUT_PROJECT.exists(), f"project does not exist: {INPUT_PROJECT}"


match metashape.load_project(INPUT_PROJECT):
    case Ok(path):
        logger.info(path)
    case Err(error_message):
        logger.info(error_message)
    case _:
        raise NotImplementedError

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
LoadProject: path = /data/kingston_snv_01/acfr_metashape_projects_dev/r234xgje_dense_with_metadata.psz


2024-11-11 12:06:47.837 | INFO     | __main__:<module>:28 - loaded document /data/kingston_snv_01/acfr_metashape_projects_dev/r234xgje_dense_with_metadata.psz successfully


loaded project in 20.163 sec


In [2]:
from mynd.collections import GroupID
from mynd.geometry import PointCloudLoader, PointCloudProcessor
from mynd.registration import (
    RegistrationPipeline,
    build_registration_pipeline,
)

from mynd.io import read_config
from mynd.utils.log import logger


CONFIG_FILE: Path = Path(
    "/home/martin/dev/mynd/config/registration_advanced.toml"
)

config: dict = read_config(CONFIG_FILE).unwrap()
registration_config: dict = config.get("registration")

logger.info(registration_config.get("aligner"))

for module in registration_config.get("refiner"):
    logger.info(module)

pipeline: RegistrationPipeline = build_registration_pipeline(
    config.get("registration")
)

# Retrieve dense point clouds
retrieval_result: Result = metashape.dense_services.retrieve_dense_point_clouds(
    cache=CACHE, overwrite=False
)

if retrieval_result.is_err():
    logger.error(retrieval_result.err())

point_cloud_loaders: dict[GroupID, PointCloudLoader] = retrieval_result.ok()

2024-11-11 12:06:47.854 | INFO     | __main__:<module>:19 - {'name': '01_aligner', 'type': 'feature_ransac', 'preprocessor': {'downsample': {'spacing': 0.2}, 'estimate_normals': {'radius': 0.4, 'neighbours': 30}}, 'matcher': {'feature': {'radius': 2.0, 'neighbours': 200}, 'point_to_point': {'with_scaling': True}, 'validators': {'distance_threshold': 0.15, 'edge_threshold': 0.95, 'normal_threshold': 5.0}, 'convergence': {'max_iteration': 50000000, 'confidence': 1.0}, 'algorithm': {'distance_threshold': 0.15, 'sample_count': 3, 'mutual_filter': True}}}
2024-11-11 12:06:47.855 | INFO     | __main__:<module>:22 - {'name': '02_coarse_colored_icp', 'type': 'colored_icp', 'preprocessor': {'downsample': {'spacing': 0.2}, 'estimate_normals': {'radius': 0.4, 'neighbours': 30}}, 'matcher': {'colored_icp_estimation': {'lambda_geometric': 0.968}, 'huber_kernel': {'k': 0.4}, 'convergence_criteria': {'relative_fitness': 1e-06, 'relative_rmse': 1e-06, 'max_iteration': 50}, 'distance_threshold': 0.2}}


In [3]:
from mynd.geometry import PointCloud

from mynd.registration import RegistrationResult
from mynd.registration import RegistrationIndex, generate_cascade_indices
from mynd.registration import RegistrationBatch, register_batch

from mynd.registration import log_registration_result
from mynd.visualization import visualize_registration


def callback_registration(
    target: PointCloud, source: PointCloud, result: RegistrationResult
) -> None:
    """Callback for registration."""
    logger.info("")
    log_registration_result(result=result)
    logger.info("")


batch: RegistrationBatch = RegistrationBatch[GroupID](point_cloud_loaders)

indices: list[RegistrationIndex] = generate_cascade_indices(batch.keys())

logger.info("Performing batch registration...")
registration_results: list[RegistrationBatch.PairResult] = register_batch(
    batch, pipeline, indices, callback=callback_registration
)
logger.info("Batch registration done!")

2024-11-11 12:06:47.882 | INFO     | __main__:<module>:24 - Performing batch registration...
2024-11-11 12:07:14.799 | INFO     | __main__:callback_registration:15 - 
2024-11-11 12:07:14.800 | INFO     | mynd.registration.utilities:log_registration_result:52 - Corresp.:     23715
2024-11-11 12:07:14.800 | INFO     | mynd.registration.utilities:log_registration_result:53 - Fitness:      0.63429
2024-11-11 12:07:14.803 | INFO     | mynd.registration.utilities:log_registration_result:54 - Inlier RMSE:  0.08817
2024-11-11 12:07:14.804 | INFO     | mynd.registration.utilities:log_registration_result:55 - Scale:        0.973
2024-11-11 12:07:14.804 | INFO     | mynd.registration.utilities:log_registration_result:56 - Translation:  [-9.23  -1.392 -0.376]
2024-11-11 12:07:14.804 | INFO     | mynd.registration.utilities:log_registration_result:57 - Rot. ZYX:     -0.74, -0.06, -0.33
2024-11-11 12:07:14.805 | INFO     | __main__:callback_registration:17 - 
2024-11-11 12:07:16.135 | INFO     | __m

### Get consensus estimate with pose graph optimization

In [4]:
# TODO: Build pose graph
# TODO: Optimize pose graph

### Visualize registration results

In [5]:
visualize: bool = False
if visualize:
    for registration in registration_results:
        target_loader: PointCloudLoader = batch.get(registration.target)
        source_loader: PointCloudLoader = batch.get(registration.source)

        target_cloud: PointCloud = target_loader().unwrap()
        source_cloud: PointCloud = source_loader().unwrap()

        visualize_registration(
            target=target_cloud,
            source=source_cloud,
            transformation=registration.result.transformation,
        )

### Update chunk transforms

In [6]:
import copy

import Metashape as ms
import numpy as np


chunks: dict[GroupID, ms.Chunk] = {
    GroupID(chunk.key, chunk.label): chunk
    for chunk in metashape.context._backend_data.get("document").chunks
}

results: dict[GroupID, dict[GroupID, RegistrationResult]] = dict()

for registration in registration_results:
    if registration.target not in results:
        results[registration.target] = dict()

    results[registration.target][registration.source] = registration.result

target: GroupID = GroupID(key=0, label="r234xgje_20100604_230524")
results: dict[GroupID, RegistrationResult] = results.get(target)


target_chunk: ms.Chunk = chunks.get(target)
for source, registration in results.items():
    source_chunk: ms.Chunk = chunks.get(source)

    logger.info(
        "Aligning chunks - target: {target_chunk.label}, source: {source_chunk.label}"
    )
    metashape.align_chunks(target_chunk, source_chunk, registration)

2024-11-11 12:10:08.461 | INFO     | __main__:<module>:28 - Aligning chunks - target: {target_chunk.label}, source: {source_chunk.label}
2024-11-11 12:10:08.461 | INFO     | __main__:<module>:28 - Aligning chunks - target: {target_chunk.label}, source: {source_chunk.label}


### TODO: Save project to file

In [7]:
assert (
    INPUT_PROJECT != DESTINATION_PROJECT
), "source and destination project cannot be the same!"

match metashape.save_project(DESTINATION_PROJECT):
    case Ok(path):
        logger.info(f"saved project to: {path}")
    case Err(message):
        logger.error(message)
    case _:
        raise NotImplementedError

SaveProject: path = /data/kingston_snv_01/acfr_metashape_projects_registered/r234xgje_registered_with_metadata.psz


2024-11-11 12:10:46.109 | INFO     | __main__:<module>:5 - saved project to: /data/kingston_snv_01/acfr_metashape_projects_registered/r234xgje_registered_with_metadata.psz


saved project in 37.5203 sec
